#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [5]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_2_4')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/import'

# Specify the filenames to move
filenames = ['spectra.csv', 'peakrange.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [6]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/bin'

# Construct the command
# command = [
#     './bin/neo4j-admin', 'database', 'import', 'full',
#     '--nodes=import/nodes.csv',
#     '--relationships=import/relationships.csv', 'neo4j'
# ]

command = 'neo4j-admin database import full --nodes=import/spectra.csv --nodes=import/peakrange.csv --relationships=import/relationships.csv --overwrite-destination neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)
#result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
Neo4j version: 5.12.0
Importing the contents of these files into C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\data\databases\neo4j:
Nodes:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\spectra.csv
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\peakrange.csv

Relationships:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\relationships.csv


Available resources:
  Total machine memory: 31.86GiB
  Free machine memory: 22.16GiB
  Max heap memory : 910.5MiB
  Max worker threads: 8
  Configured max memory: 19.23GiB
  High parallel IO: true

Cypher type normalization is enabled (disable with --normalize-types=false):
  Property type of 'Weight' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\rel

Run and connect to the Neo4j Database

In [9]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [10]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['PeakRange', 'Spectra'],
      {
        Has_Peak: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [11]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, score
    ORDER BY score DESC, SpecID ASC, PeakRange ASC
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["score"]) for record in results]

In [12]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, score
    ORDER BY score DESC, SpecID ASC, PeakRange ASC
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["score"]) for record in results]

In [13]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, score
    ORDER BY score DESC, SpecID ASC, PeakRange ASC
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["score"]) for record in results]

In [14]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, score
    ORDER BY score DESC, SpecID ASC, PeakRange ASC
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["score"]) for record in results]

In [15]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, Community
    ORDER BY Community, PeakRange, SpecID
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["Community"]) for record in results]

In [16]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, Community
    ORDER BY Community, PeakRange, SpecID
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["Community"]) for record in results]

In [17]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, Community
    ORDER BY Community, PeakRange, SpecID
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["Community"]) for record in results]

In [13]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [51]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS SpecID, gds.util.asNode(nodeId).PeakRange AS PeakRange, embedding
    ORDER BY SpecID ASC, PeakRange ASC
    """
    results = tx.run(query)
    return [(record["SpecID"], record["PeakRange"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [24]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [25]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['SpecID', 'PeakRange', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['SpecID', 'PeakRange', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['SpecID', 'PeakRange', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['SpecID', 'PeakRange', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['SpecID', 'PeakRange', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['SpecID', 'PeakRange', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['SpecID', 'PeakRange', 'Louvain'])

In [30]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['SpecID', 'PeakRange'], how='left')

In [31]:
df = merged_df
df

,SpecID,PeakRange,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,None,1542-1594,206.305103,1901.248561,0.480389,11.445448,10,555
1,None,407-495,189.096232,1792.443871,0.453042,10.763114,6,552
2,None,603-738,181.013801,1711.208851,0.432480,10.283880,3,552
3,None,1212-1294,138.831768,1295.154804,0.327270,7.828487,14,560
4,None,1427-1446,108.825565,1039.193580,0.262582,6.297912,6,552
...,...,...,...,...,...,...,...,...
3056,210225-2-16,None,0.193766,0.499277,0.000126,0.152476,10,555
3057,210217-1-46,None,0.187232,0.418843,0.000107,0.152115,10,555
3058,210218-1-01,None,0.185530,0.401657,0.000103,0.152028,6,552
3059,210303-2-26,None,0.180093,0.335550,0.000086,0.151703,10,560


In [53]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'PeakRange', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,PeakRange,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,None,0.051867,0.158488,0.086045,0.003812,0.066507,-0.224245,-0.122220,-0.052023,...,-0.062555,0.006362,-0.097876,-0.112830,0.063908,-0.122438,0.136933,-0.076990,-0.325940,-0.195051
1,201210-1-01,None,0.053686,0.151142,0.087149,0.006063,0.060831,-0.226891,-0.121547,-0.047822,...,-0.074486,0.005000,-0.082877,-0.117371,0.053649,-0.125731,0.119779,-0.072205,-0.327756,-0.189015
2,201210-1-02,None,0.049516,0.144117,0.072947,0.008066,0.070512,-0.207010,-0.129483,-0.053965,...,-0.070224,-0.007337,-0.086669,-0.093649,0.060685,-0.127946,0.121677,-0.083342,-0.331808,-0.196491
3,201210-1-03,None,0.023108,0.140536,0.091474,0.005163,0.049884,-0.217187,-0.124632,-0.047186,...,-0.064930,0.014801,-0.090947,-0.086368,0.068546,-0.144794,0.136578,-0.072240,-0.328465,-0.173267
4,201210-1-04,None,0.051534,0.158593,0.094775,-0.004096,0.049779,-0.221590,-0.119742,-0.049362,...,-0.080999,-0.008604,-0.082310,-0.116915,0.038921,-0.121625,0.107469,-0.068101,-0.339703,-0.184405


Delete the projection

In [20]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [21]:
close_driver()

In [37]:
status_df = pd.read_csv('../../data/graph_2_3/nodes.csv')

In [38]:
status_df.drop(columns=['Unnamed: 0'], inplace=True)

In [39]:
status_df

,SpecID:ID,Status:LABEL,SurID
0,201210-1-00,Normal,201210-1
1,201210-1-01,Normal,201210-1
2,201210-1-02,Normal,201210-1
3,201210-1-03,Normal,201210-1
4,201210-1-04,Normal,201210-1
...,...,...,...
3040,210526-3-45,Hyperglycemia,210526-3
3041,210526-3-46,Hyperglycemia,210526-3
3042,210526-3-47,Hyperglycemia,210526-3
3043,210526-3-48,Hyperglycemia,210526-3


In [46]:
graph_df = pd.merge(df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
graph_df.drop(columns=["SpecID:ID", "PeakRange"], inplace=True)

In [56]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
fast_graph_df.drop(columns=["SpecID:ID", "PeakRange"], inplace=True)

In [47]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [57]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [58]:
fast_graph_df

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status,SurID
0,201210-1-00,0.051867,0.158488,0.086045,0.003812,0.066507,-0.224245,-0.122220,-0.052023,0.169292,...,-0.097876,-0.112830,0.063908,-0.122438,0.136933,-0.076990,-0.325940,-0.195051,Normal,201210-1
1,201210-1-01,0.053686,0.151142,0.087149,0.006063,0.060831,-0.226891,-0.121547,-0.047822,0.163328,...,-0.082877,-0.117371,0.053649,-0.125731,0.119779,-0.072205,-0.327756,-0.189015,Normal,201210-1
2,201210-1-02,0.049516,0.144117,0.072947,0.008066,0.070512,-0.207010,-0.129483,-0.053965,0.149684,...,-0.086669,-0.093649,0.060685,-0.127946,0.121677,-0.083342,-0.331808,-0.196491,Normal,201210-1
3,201210-1-03,0.023108,0.140536,0.091474,0.005163,0.049884,-0.217187,-0.124632,-0.047186,0.145268,...,-0.090947,-0.086368,0.068546,-0.144794,0.136578,-0.072240,-0.328465,-0.173267,Normal,201210-1
4,201210-1-04,0.051534,0.158593,0.094775,-0.004096,0.049779,-0.221590,-0.119742,-0.049362,0.155087,...,-0.082310,-0.116915,0.038921,-0.121625,0.107469,-0.068101,-0.339703,-0.184405,Normal,201210-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210526-3-45,0.064683,0.108682,0.063354,0.018714,0.067830,-0.223698,-0.121989,-0.049586,0.170442,...,-0.081682,-0.133928,0.061188,-0.109800,0.147087,-0.050465,-0.345352,-0.180106,Hyperglycemia,210526-3
3041,210526-3-46,0.055027,0.142254,0.084567,0.028125,0.094218,-0.242654,-0.096311,-0.054721,0.187980,...,-0.100080,-0.113410,0.066125,-0.116915,0.159479,-0.075905,-0.330651,-0.200743,Hyperglycemia,210526-3
3042,210526-3-47,0.052272,0.123678,0.051036,0.017032,0.073423,-0.248813,-0.125230,-0.047890,0.159602,...,-0.086636,-0.118247,0.063513,-0.116144,0.151278,-0.060860,-0.340582,-0.180533,Hyperglycemia,210526-3
3043,210526-3-48,0.058758,0.139276,0.080845,0.028339,0.078330,-0.235466,-0.105315,-0.046183,0.180169,...,-0.091590,-0.122065,0.071529,-0.115867,0.157437,-0.067143,-0.326524,-0.190702,Hyperglycemia,210526-3


In [45]:
graph_df

,SpecID,PeakRange,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,SurID
0,201210-2-23,None,0.988151,9.523999,0.002442,0.198178,6,546,Normal,201210-2
1,201210-2-09,None,0.937695,8.958040,0.002297,0.195252,14,547,Normal,201210-2
2,201210-2-24,None,0.936987,8.976078,0.002301,0.195357,6,546,Normal,201210-2
3,201210-2-08,None,0.898231,8.478316,0.002173,0.193094,16,546,Normal,201210-2
4,201210-2-01,None,0.896629,8.483619,0.002173,0.192876,6,546,Normal,201210-2
...,...,...,...,...,...,...,...,...,...,...
3040,210225-2-16,None,0.193766,0.499277,0.000126,0.152476,10,555,Hypoglycemia,210225-2
3041,210217-1-46,None,0.187232,0.418843,0.000107,0.152115,10,555,Hyperglycemia,210217-1
3042,210218-1-01,None,0.185530,0.401657,0.000103,0.152028,6,552,Hyperglycemia,210218-1
3043,210303-2-26,None,0.180093,0.335550,0.000086,0.151703,10,560,Normal,210303-2


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

def evaluate_extra_trees(df):

    # Set the Surfaces as groups
    groups = graph_df['SurID']
    X = graph_df.drop(['Status', 'SurID', 'SpecID'], axis=1)
    y = graph_df['Status']

    # Creating the Extra Trees classifier
    et = ExtraTreesClassifier(random_state=1234)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    scores = []
    for train_index, test_index in cv.split(X, y, groups):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the Extra Trees classifier
        et.fit(X_train, y_train)
        predictions = et.predict(X_test)
        
        # Evaluate the model
        score = accuracy_score(y_test, predictions)
        scores.append(score)
    
    # Displaying the results
    print(f'{et.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

In [50]:
evaluate_extra_trees(graph_df)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3515 +/- 0.0261


In [59]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = graph_df.drop(['Status', 'SpecID', 'SurID'], axis=1)
y = graph_df['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.4128 +/- 0.0320
ExtraTreesClassifier Cross-Validation Accuracy: 0.4069 +/- 0.0242


In [60]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [61]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [62]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [63]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.39408866995073893


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.38      0.36      0.37       187
 Hypoglycemia       0.43      0.45      0.44       210
       Normal       0.37      0.37      0.37       212

     accuracy                           0.39       609
    macro avg       0.39      0.39      0.39       609
 weighted avg       0.39      0.39      0.39       609


Confusion Matrix:
[[68 50 69]
 [51 94 65]
 [60 74 78]]


In [64]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.3973727422003284


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.39      0.39      0.39       187
 Hypoglycemia       0.42      0.40      0.41       210
       Normal       0.38      0.40      0.39       212

     accuracy                           0.40       609
    macro avg       0.40      0.40      0.40       609
 weighted avg       0.40      0.40      0.40       609


Confusion Matrix:
[[72 48 67]
 [52 85 73]
 [59 68 85]]


In [65]:
get_feature_importances(rf_model, X)

,Feature,Importance
1,DegreeCentrality,0.219316
0,PageRank,0.217476
3,ArticleRank,0.212739
2,EigenvectorCentrality,0.210966
5,Louvain,0.077684
4,Leiden,0.061820


In [66]:
get_feature_importances(et_model, X)

,Feature,Importance
0,PageRank,0.221346
1,DegreeCentrality,0.215508
3,ArticleRank,0.213972
2,EigenvectorCentrality,0.212154
5,Louvain,0.075499
4,Leiden,0.061522


## FastRp

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['SpecID', 'Status', 'SurID'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.5927750410509032
Extra Trees Accuracy: 0.6026272577996716


In [72]:
evaluate_extra_trees(fast_graph_df)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3515 +/- 0.0261


Fast rp mixed with traditional graph metrics

In [68]:
joined_graph = pd.merge(graph_df, fast_graph_df, on=['SpecID', 'Status', 'SurID'], how='inner')

In [69]:
joined_graph

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,SurID,embedding_0,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-2-23,0.988151,9.523999,0.002442,0.198178,6,546,Normal,201210-2,0.063174,...,-0.084844,0.023734,-0.097987,-0.129791,0.060849,-0.118618,0.140332,-0.065260,-0.332914,-0.189996
1,201210-2-09,0.937695,8.958040,0.002297,0.195252,14,547,Normal,201210-2,0.064894,...,-0.064918,0.019445,-0.092588,-0.117841,0.053145,-0.117667,0.133723,-0.082518,-0.344542,-0.178393
2,201210-2-24,0.936987,8.976078,0.002301,0.195357,6,546,Normal,201210-2,0.059709,...,-0.073967,0.018379,-0.093740,-0.128696,0.052466,-0.108429,0.135364,-0.062878,-0.337085,-0.188664
3,201210-2-08,0.898231,8.478316,0.002173,0.193094,16,546,Normal,201210-2,0.059520,...,-0.072411,0.017678,-0.088200,-0.128865,0.054680,-0.108560,0.112801,-0.075402,-0.319978,-0.192468
4,201210-2-01,0.896629,8.483619,0.002173,0.192876,6,546,Normal,201210-2,0.068262,...,-0.070107,0.028741,-0.097141,-0.117560,0.068625,-0.124433,0.143035,-0.079431,-0.331429,-0.182562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210225-2-16,0.193766,0.499277,0.000126,0.152476,10,555,Hypoglycemia,210225-2,0.047838,...,-0.058531,0.057914,-0.090824,-0.127141,0.085306,-0.143878,0.159739,-0.091060,-0.315711,-0.184502
3041,210217-1-46,0.187232,0.418843,0.000107,0.152115,10,555,Hyperglycemia,210217-1,0.033462,...,-0.067329,0.017309,-0.085133,-0.106121,0.066759,-0.148660,0.132093,-0.086525,-0.327887,-0.174715
3042,210218-1-01,0.185530,0.401657,0.000103,0.152028,6,552,Hyperglycemia,210218-1,0.039471,...,-0.065934,0.036691,-0.092677,-0.109834,0.075627,-0.129605,0.156662,-0.074169,-0.332824,-0.183661
3043,210303-2-26,0.180093,0.335550,0.000086,0.151703,10,560,Normal,210303-2,0.082661,...,-0.028196,0.073403,-0.129958,-0.099339,0.056472,-0.112851,0.135046,-0.122110,-0.321515,-0.215795


In [70]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status', 'SurID'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.5878489326765188
Extra Trees Accuracy: 0.6157635467980296


In [71]:
evaluate_extra_trees(joined_graph)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3515 +/- 0.0261
